In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
#import pickle
#import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
#import seaborn as sns
#from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
#from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.decomposition import PCA
import glob
#import sys
#np.set_printoptions(threshold=sys.maxsize)

2022-12-09 18:15:01.286539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 18:15:01.286589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dados NSL-KDD

In [2]:
#Carrega dados do dataset
#monday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Monday-WorkingHours.csv", skipinitialspace = True)
#tuesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Tuesday-WorkingHours.csv", skipinitialspace = True)
#wednesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Wednesday-WorkingHours.csv", skipinitialspace = True)
#thursday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Thursday-WorkingHours.csv", skipinitialspace = True)
#friday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Friday-WorkingHours.csv", skipinitialspace = True)

# Carregamento Datasets

In [3]:
#Carrega dados do dataset
monday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Monday-WorkingHours.csv", skipinitialspace = True)
tuesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Tuesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((monday, tuesday), axis=0)
del monday, tuesday

In [4]:
wednesday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Wednesday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, wednesday), axis=0)
del wednesday

In [5]:
thursday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Thursday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, thursday), axis=0)
del thursday

In [6]:
friday = pd.read_csv(r"CIC-IDS2017 ATUALIZADO/Friday-WorkingHours.csv", skipinitialspace = True)
x = pd.concat((x, friday), axis=0)
del friday

In [7]:
#Concatena todos os datasets carregados
#x = pd.concat((monday, tuesday, wednesday, thursday, friday), axis=0)
x = x.drop(['Flow ID', 'Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp'], axis = 1)

In [8]:
#Transforma o dataset multiclasse para binário
x['Label'] = x['Label'].replace('BENIGN', 0)
x['Label'] = x['Label'].replace('FTP-Patator', 1)
x['Label'] = x['Label'].replace('SSH-Patator', 1)
x['Label'] = x['Label'].replace('FTP-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('SSH-Patator - Attempted', 0)
x['Label'] = x['Label'].replace('DoS slowloris', 1)
x['Label'] = x['Label'].replace('DoS slowloris - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Slowhttptest', 1)
x['Label'] = x['Label'].replace('DoS Slowhttptest - Attempted', 0)
x['Label'] = x['Label'].replace('DoS Hulk', 1)
x['Label'] = x['Label'].replace('DoS Hulk - Attempted', 0)
x['Label'] = x['Label'].replace('DoS GoldenEye', 1)
x['Label'] = x['Label'].replace('Heartbleed', 1)
x['Label'] = x['Label'].replace('DoS GoldenEye - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Brute Force', 1)
x['Label'] = x['Label'].replace('Web Attack - Brute Force - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - XSS', 1)
x['Label'] = x['Label'].replace('Web Attack - XSS - Attempted', 0)
x['Label'] = x['Label'].replace('Web Attack - Sql Injection', 1)
x['Label'] = x['Label'].replace('Infiltration', 1)
x['Label'] = x['Label'].replace('Infiltration - Attempted', 0)
x['Label'] = x['Label'].replace('Bot', 1)
x['Label'] = x['Label'].replace('Bot - Attempted', 0)
x['Label'] = x['Label'].replace('DDoS', 1)
x['Label'] = x['Label'].replace('PortScan', 1)

In [9]:
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.dropna(inplace=True)

In [10]:
#Realiza o One-Hot Encoding
#x = pd.get_dummies(x)

In [11]:
#Separa as labels e os Dados em datasets diferentes
data_class = x['Label']
data = x.drop(['Label'], axis = 1) 

In [12]:
data = data.to_numpy()
#transforma dados infinitos em finitos
#data = np.nan_to_num(data,posinf=10000000, neginf=10000000)
data_class = data_class.to_numpy()

In [13]:
noTrain = np.count_nonzero(data_class == 0)
anTrain = np.count_nonzero(data_class == 1)

In [14]:
print("Dados Normais: ", noTrain, "\nDados Anômalos: ", anTrain)

Dados Normais:  1666172 
Dados Anômalos:  433849


# Abordagem Autoencoder-KNN

In [15]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    print("Fold ",fold_no)
    #####################################################
    ####Pré-Processamento
    print("Pré-processando...")
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(anTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, noTrain)) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    autoencoderTrain_class = np.zeros(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####PCA
    print("PCA")
    #Modela o PCA
    var = autoencoderTrain.shape[1]
    pca = PCA(n_components=int(var/4))
         
    #####################################################
    ####KNN
    ###Treinamento KNN
    print("KNN...")
    pca.fit(autoencoderTrain)
    pca_encoder = pca.transform(knnTrain)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(pca_encoder, knnTrain_class)
    
    #Teste do Modelo
    pca_encoder = pca.transform(modelTest)
    predict_knn = neigh.predict(pca_encoder)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(modelTest_class, predict_knn[:])
    recall_knn[count] = recall_score(modelTest_class, predict_knn[:])
    precision_knn[count] = precision_score(modelTest_class, predict_knn[:])
    f1_knn[count] = f1_score(modelTest_class, predict_knn[:])
    matrix_knn[count] = confusion_matrix(modelTest_class, predict_knn[:])
    print("Finalizado.")
   
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    print("DT...")
    pca_encoder = pca.transform(knnTrain)
    dt = tree.DecisionTreeClassifier()
    dt.fit(pca_encoder, knnTrain_class)

    #Teste
    pca_encoder = pca.transform(modelTest)
    predict_dt = dt.predict(pca_encoder)

    #Métricas
    accuracy_dt[count] = accuracy_score(modelTest_class, predict_dt[:])
    recall_dt[count] = recall_score(modelTest_class, predict_dt[:])
    precision_dt[count] = precision_score(modelTest_class, predict_dt[:])
    f1_dt[count] = f1_score(modelTest_class, predict_dt[:])
    matrix_dt[count] = confusion_matrix(modelTest_class, predict_dt[:])
    print("Finalizado.")
    
    #####################################################
    ####SVM
    #Treinamento SVM
    print("SVM...")
    pca_encoder = pca.transform(knnTrain)
    clf = svm.SVC(kernel='rbf')
    clf.fit(pca_encoder, knnTrain_class)
    
    #Teste
    pca_encoder = pca.transform(modelTest)
    predict_svm = clf.predict(pca_encoder)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(modelTest_class, predict_svm[:])
    recall_svm[count] = recall_score(modelTest_class, predict_svm[:])
    precision_svm[count] = precision_score(modelTest_class, predict_svm[:])
    f1_svm[count] = f1_score(modelTest_class, predict_svm[:])
    matrix_svm[count] = confusion_matrix(modelTest_class, predict_svm[:])
    print("Finalizado.")
    
    Result_GridSearchLimiar = np.array(["KNN", "Acurácia", accuracy_knn[count], "",
                                        "Recall", recall_knn[count], "",
                                        "Precision", precision_knn[count], "",
                                        "F1", f1_knn[count], "-----------------","-----------------",
                                    "Decision Tree",  "Acurácia", accuracy_dt[count],"",
                                        "Recall", recall_dt[count], "",
                                        "Precision", precision_dt[count], "",
                                        "F1", f1_dt[count], "-----------------","-----------------",
                                    "SVM",  "Acurácia", accuracy_svm[count],"",
                                        "Recall", recall_svm[count], "",
                                        "Precision", precision_svm[count], "",
                                        "F1", f1_svm[count], "-----------------","-----------------","-----------------","-----------------",
                                    "KNN", matrix_knn[count], "-----------------","-----------------",
                                    "Decision Tree", matrix_dt[count], "-----------------","-----------------",
                                    "SVM", matrix_svm[count]])

    np.savetxt('Result_Parcial_CICIDS2017_PCA_Fold'+str(fold_no)+'.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')
    
    count = count + 1
    fold_no = fold_no + 1

Fold  1
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  2
Pré-processando...


/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  3
Pré-processando...


/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  4
Pré-processando...


...

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  10
Pré-processando...


/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.


/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [16]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[1.651884e+05 1.428800e+03]
 [7.800000e+01 4.330690e+04]] 
Acurácia:  0.9928248339811196  +/-  0.00040375780336010297 
Recall:  0.9982021365318744  +/-  0.0008424984458127966 
Precision:  0.9680691193909123  +/-  0.002425010224733569 
F1:  0.9829021424457676  +/-  0.0009348031797075339

Autoencoder - SVM:
 [[163214.8   3402.4]
 [   164.   43220.9]] 
Acurácia:  0.98301731479867  +/-  0.005415432741519281 
Recall:  0.9962198822638122  +/-  0.00043818836998161777 
Precision:  0.927567339531206  +/-  0.022563349611918426 
F1:  0.9605286972183716  +/-  0.012160833813825518

Autoencoder - Decision Tree:
 [[1.650263e+05 1.590900e+03]
 [2.130000e+01 4.336360e+04]] 
Acurácia:  0.9923229339810746  +/-  0.00018285251228485942 
Recall:  0.9995090457899056  +/-  0.00011434467619755388 
Precision:  0.964611562295788  +/-  0.0008117908793653536 
F1:  0.9817501095023016  +/-  0.00042714076211556446


In [17]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_PCA.txt', Result, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == "":


In [18]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_PCA_Matrix.txt', Result_Matrix, delimiter = ',',fmt='%s')

/home/murilo/anaconda3/envs/luan/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
